#### Using Descriptors and Hybrids

A _more comprehensive way_ to produce _modified behavior_ for an attribute is __to use descriptors__. These are commonly used in Python using the `property()` function. The standard SQLAlchemy technique for descriptors is to __create a plain descriptor__, and to have it _read/write from a mapped attribute with a different name_. Below we illustrate this using Python 2.6-style properties.

In [1]:
from sqlalchemy import Column, Integer, String, func
from sqlalchemy.orm import declarative_base
from sqlalchemy.ext.hybrid import hybrid_property

In [2]:
Base = declarative_base()

In [3]:
class EmailAddressWithProperty(Base):
    __tablename__ = "email_address"
    
    id = Column(Integer, primary_key=True)
    # name the attribute with an underscore, different from the column name
    _email = Column("email", String)
    
    # then create an ".email" attribute, to get/set "._email"
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, email):
        self._email = email

The approach above will work, but there's more we can add. While our `EmailAddressWithProperty` object will __shuttle the value__ through the _email descriptor_ and into the *_email mapped attribute*, the _class level_ `EmailAddressWithProperty.email` _attribute_ __does not have the usual expression semantics usable with `Query`__. To provide these, we instead use the `hybrid extension`.

In [4]:
class EmailAddressHybridExtension(Base):
    __tablename__ = "email_address_hybrid"
    
    id = Column(Integer, primary_key=True)
    _email = Column("email", String)
    
    @hybrid_property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, email):
        self._email = email

The `.email` attribute, in addition to providing _getter/setter behavior_ when we have an instance of `EmailAddressHybridExtension`, also _provides a SQL expression_ when __used at the class level__, that is, from the `EmailAddressHybridExtension` class __directly__.

```
address = (
    session.query(EmailAddressHybridExtension).
    filter(
        EmailAddressHybridExtension.email == "address@example.com"
    ).one()
)
```

The `hybrid_property` also allows us to __change the behavior of the attribute__, including _defining separate behaviors_ when the attribute is _accessed at the instance level_ versus at the _class/expression level_, using the `hybrid_property.expression()` _modifier_. Such as, if we wanted to _add a host name automatically_, we might define __two sets of string manipulation logic__.

In [5]:
class EmailAddressPropertyExpression(Base):
    __tablename__ = "email_address_expression"
    
    id = Column(Integer, primary_key=True)
    _email = Column("email", String)
    
    @hybrid_property
    def email(self):
        """Return the value of _email up until the last twelve characters."""
        return self._email[:-12]
    
    @email.setter
    def email(self, email):
        """Set the value of _email, tacking on the twelve character value @example.com."""
        self._email = f"{email}@example.com"
    
    @email.expression
    def email(cls):
        """Produce a SQL expression that represents the value
        of the _email column, minus the last twelve characters."""
        return func.substr(cls._email, 0, func.length(cls._email) - 12)

Above, accessing the _email_ property of an instance of `EmailAddressPropertyExpression` will return the value of the *_email* attribute, __removing or adding the hostname__ `@example.com` from the value. When we _query against the email attribute_, a __SQL function is rendered__ which produces the _same effect_.

```
sqladdress = (
    session.query(EmailAddressPropertyExpression).
    filter(EmailAddressPropertyExpression.email == "address").one()
)
```